In [11]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import streamlit as st

# Load documents
loader = TextLoader("speech.txt")
docs = loader.load()
print(docs)  # Optional: just to see the loaded docs

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document = text_splitter.split_documents(docs)

# Vectorstore with Ollama Embeddings
db = Chroma.from_documents(document[:20], OllamaEmbeddings(model="llama2"))

# Setup LLM
llm = Ollama(model="llama2")

# Setup Prompt Template
prompt = ChatPromptTemplate.from_template("""
answer the following question based on the provided context. Think carefully before answering, it's very high-quality data.
<context>{context}</context>
Question: {input}
""")

# Create chains
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Query the chain
def data_question(question):
    return retrieval_chain.invoke({"input":{question}})
st.title("RAG Question Answering base on Text file")
question = st.text_input("Please enter your question:")
if question:
    st.write(data_question(question))

[Document(metadata={'source': 'speech.txt'}, page_content='War Message to Congress\nBy Woodrow Wilson\nApril 2, 1917\n\nGentlemen of the Congress:\n\nI have called the Congress into extraordinary session because there are serious, very serious, choices of policy to be made, and it is necessary that we should have a clear and definite understanding as to the matters that are to be dealt with and the way in which they are to be handled.\n\nOn the 3rd of February last, I officially laid before you the extraordinary announcement of the Imperial German Government that on and after the 1st day of February it was its purpose to put aside all restraints of law or of humanity and use its submarines to sink every vessel that sought to approach either the ports of Great Britain and Ireland or the western coasts of Europe or any of the ports controlled by the enemies of Germany within the Mediterranean.\n\nThat policy has been followed with unrelenting severity.\n\nIt is a war against all nations.

{'input': 'What data you has',
 'context': [Document(metadata={'source': 'speech.txt'}, page_content='It will involve the bringing to the service of the government the entire financial power of the country in order that the resources of the country may be marshaled and organized in aid of the nation.\n\nIt will involve the development of an economic mobilization.\n\nIt will involve the immediate enlistment of not only of the men, but of all the energy, enthusiasm, and intelligence of the country.\n\nIt is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance.\n\nBut the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts.\n\nFor democracy, for the right of those who submit to authority to have a voice in their own government, for the rights and liberties of small nations, for'),
  Document(metadata={'source': 'speech